In [ ]:
#포르토 세구로 안전 운전자 예측 경진대회
#https://www.kaggle.com/c/porto-seguro-safe-driver-prediction
#운전자A가 내년에 보험을 청구할 확률을 예측하는 경진대회
#학습 데이터는 총 59만, 테스트 데이터는 총 89만개의 운전자 데이터를 포함

In [ ]:
#전처리로 StandardScaler, PCA사용
#신경망 모델 적용()

In [179]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [180]:
p_train = pd.read_csv('../data/porto-seguro-safe-driver-prediction/train.csv')
p_test = pd.read_csv('../data/porto-seguro-safe-driver-prediction/test.csv')

In [181]:
p_train.shape, p_test.shape

((595212, 59), (892816, 58))

In [182]:
p_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
id                595212 non-null int64
target            595212 non-null int64
ps_ind_01         595212 non-null int64
ps_ind_02_cat     595212 non-null int64
ps_ind_03         595212 non-null int64
ps_ind_04_cat     595212 non-null int64
ps_ind_05_cat     595212 non-null int64
ps_ind_06_bin     595212 non-null int64
ps_ind_07_bin     595212 non-null int64
ps_ind_08_bin     595212 non-null int64
ps_ind_09_bin     595212 non-null int64
ps_ind_10_bin     595212 non-null int64
ps_ind_11_bin     595212 non-null int64
ps_ind_12_bin     595212 non-null int64
ps_ind_13_bin     595212 non-null int64
ps_ind_14         595212 non-null int64
ps_ind_15         595212 non-null int64
ps_ind_16_bin     595212 non-null int64
ps_ind_17_bin     595212 non-null int64
ps_ind_18_bin     595212 non-null int64
ps_reg_01         595212 non-null float64
ps_reg_02         595212 non-null float64
ps_re

In [183]:
p_train.isna().sum()

id                0
target            0
ps_ind_01         0
ps_ind_02_cat     0
ps_ind_03         0
ps_ind_04_cat     0
ps_ind_05_cat     0
ps_ind_06_bin     0
ps_ind_07_bin     0
ps_ind_08_bin     0
ps_ind_09_bin     0
ps_ind_10_bin     0
ps_ind_11_bin     0
ps_ind_12_bin     0
ps_ind_13_bin     0
ps_ind_14         0
ps_ind_15         0
ps_ind_16_bin     0
ps_ind_17_bin     0
ps_ind_18_bin     0
ps_reg_01         0
ps_reg_02         0
ps_reg_03         0
ps_car_01_cat     0
ps_car_02_cat     0
ps_car_03_cat     0
ps_car_04_cat     0
ps_car_05_cat     0
ps_car_06_cat     0
ps_car_07_cat     0
ps_car_08_cat     0
ps_car_09_cat     0
ps_car_10_cat     0
ps_car_11_cat     0
ps_car_11         0
ps_car_12         0
ps_car_13         0
ps_car_14         0
ps_car_15         0
ps_calc_01        0
ps_calc_02        0
ps_calc_03        0
ps_calc_04        0
ps_calc_05        0
ps_calc_06        0
ps_calc_07        0
ps_calc_08        0
ps_calc_09        0
ps_calc_10        0
ps_calc_11        0


In [184]:
p_train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [185]:
#0 = 결측치? 특성을 알 수 없으니 결측치 대체가 어려움

In [186]:
p_train['target'].value_counts()

0    573518
1     21694
Name: target, dtype: int64

In [187]:
p_test.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1


In [188]:
y = p_train['target']

In [189]:
X_train = p_train.drop('target', axis = 1)

In [190]:
X_train.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
1,9,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
2,13,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
3,16,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0


In [191]:
X_train.shape

(595212, 58)

In [192]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_scaled = scaler.transform(X_train)

In [193]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
pca.fit(X_scaled)

X_pca = pca.transform(X_scaled)
print("원본 데이터 형태:", str(X_scaled.shape))
print("축소된 데이터 형태:", str(X_pca.shape))

원본 데이터 형태: (595212, 58)
축소된 데이터 형태: (595212, 3)


In [194]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input 

In [195]:
model= Sequential()
model.add(Dense(1280, activation='tanh', input_shape = (3, )))
model.add(Dropout(0.25))
model.add(Dense(320, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(320, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
#이진분류 모델에 적합한 시그모이드 함수 사용

In [196]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 1280)              5120      
_________________________________________________________________
dropout_19 (Dropout)         (None, 1280)              0         
_________________________________________________________________
dense_38 (Dense)             (None, 320)               409920    
_________________________________________________________________
dropout_20 (Dropout)         (None, 320)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 320)               102720    
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 321       
Total params: 518,081
Trainable params: 518,081
Non-trainable params: 0
_________________________________________________________________


In [197]:
model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam', metrics = ['accuracy'])

In [198]:
model.fit(X_pca, y, epochs=10, batch_size= 1200)

Epoch 1/10
595212/595212 [==============================] - 28s 47us/step - loss: 0.1673 - acc: 0.9595
Epoch 2/10
595212/595212 [==============================] - 26s 44us/step - loss: 0.1548 - acc: 0.9636
Epoch 3/10
595212/595212 [==============================] - 26s 43us/step - loss: 0.1544 - acc: 0.9636
Epoch 4/10
595212/595212 [==============================] - 26s 44us/step - loss: 0.1544 - acc: 0.9636
Epoch 5/10
595212/595212 [==============================] - 26s 44us/step - loss: 0.1544 - acc: 0.9636
Epoch 6/10
595212/595212 [==============================] - 26s 44us/step - loss: 0.1543 - acc: 0.9636
Epoch 7/10
595212/595212 [==============================] - 26s 43us/step - loss: 0.1542 - acc: 0.9636
Epoch 8/10
595212/595212 [==============================] - 30s 50us/step - loss: 0.1542 - acc: 0.9636
Epoch 9/10
595212/595212 [==============================] - 26s 43us/step - loss: 0.1543 - acc: 0.9636
Epoch 10/10
595212/595212 [==============================] - 26s 44us/ste

In [224]:
X_test_scaled = scaler.transform(p_test)

In [225]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
pca.fit(X_test_scaled)

X_test_scaled_pca = pca.transform(X_test_scaled)
print("원본 데이터 형태:", str(X_test_scaled.shape))
print("축소된 데이터 형태:", str(X_test_scaled_pca.shape))

원본 데이터 형태: (892816, 58)
축소된 데이터 형태: (892816, 3)


In [226]:
predictions = model.predict(X_test_scaled_pca)

In [227]:
predictions

array([[0.02391893],
       [0.03211233],
       [0.03579471],
       ...,
       [0.02751845],
       [0.0370506 ],
       [0.03561118]], dtype=float32)

In [228]:
predictions = pd.DataFrame(predictions, columns=['target'])

In [229]:
P_test = pd.read_csv('../data/porto-seguro-safe-driver-prediction/test.csv')
predictions = pd.concat((P_test.iloc[:, 0], predictions), axis = 1)

In [230]:
predictions

,id,target
0,0,0.023919
1,1,0.032112
2,2,0.035795
3,3,0.017337
4,4,0.037205
5,5,0.030582
6,6,0.023733
7,8,0.029677
8,10,0.048751
9,11,0.046206


In [231]:
predictions.to_csv('y_test_ver_2.csv', sep=",", index = False)

In [ ]:
#public score : 0.20977